In [1]:
from pyforest import *

In [2]:
# import dataset without foreign schools
df = pd.read_csv(r'/Users/dillorr/defaultRate/dillon/defaultRate/data/modeling/combined.csv', encoding = "ISO-8859-1")

<IPython.core.display.Javascript object>

# Modeling

### Train Test Split

In [7]:
# Import train_test_split()
from sklearn.model_selection import train_test_split

# Select the opeid column as the feature to be predicted (y)
y = df['proprietary']

# Remove the opeid column to create the training data
X = df.drop('proprietary', axis=1)

# Perform a 70% train and 30% test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

train = pd.concat([y_train, X_train], axis = 1)
test = pd.concat([y_test, X_test], axis = 1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# export training and test
train.to_csv('/Users/dillorr/defaultRate/dillon/defaultRate/data/modeling/train.csv', index=False)
test.to_csv('/Users/dillorr/defaultRate/dillon/defaultRate/data/modeling/test.csv', index=False)

In [8]:
train.columns

Index(['proprietary', 'opeid', 'agemedian', 'cdr2014_log', 'cdr2015_log',
       'cdr2016_log', 'cohort2014_log', 'cohort2015_log', 'cohort2016_log',
       'density_log', 'educationcollegeorabove', 'familysize_log',
       'female_sqrt', 'homeownership', 'homevalue_log',
       'incomehouseholdmedian_log', 'incomehouseholdsixfigure_log',
       'laborforceparticipation_sqrt', 'male_reciprocal', 'married',
       'population_log', 'proglength', 'raceasian_log', 'raceblack_log',
       'racemultiple_log', 'racenative_reciprocal', 'raceother_log',
       'racepacific', 'racewhite', 'rentmedian_log', 'unemploymentrate_log',
       'associates', 'bachelors', 'firstprofessional', 'gradprofessional',
       'mastersordoctors', 'nondegree', 'nondegree1year', 'nondegree2years',
       'nondegree3years', 'private', 'public', 'hbcuCollege',
       'hispanicCollege', 'nativeAmericanCollege', 'notReportedCollege'],
      dtype='object')

### Logistic Regression

In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols, glm

formula = 'proprietary ~ cdr2016_log + agemedian + density_log + familysize_log + homeownership + incomehouseholdmedian_log + laborforceparticipation_sqrt + population_log + raceasian_log + racenative_reciprocal + racewhite'


# binomial
family_GLM = sm.families.Binomial()


# Fit a linear model
model_lm = glm(formula = formula,
               data = train, family = family_GLM).fit()

# View model summary
print(model_lm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            proprietary   No. Observations:                 2693
Model:                            GLM   Df Residuals:                     2681
Model Family:                Binomial   Df Model:                           11
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1529.7
Date:                Mon, 06 Apr 2020   Deviance:                       3059.3
Time:                        20:07:05   Pearson chi2:                 2.72e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [10]:
pred_lm = model_lm.predict(X_test)

# Define the cutoff
cutoff = 0.5

# Compute class predictions: y_prediction
y_prediction = np.where(pred_lm > cutoff, 1, 0)

# Compute the confusion matrix using crosstab function
conf_mat = pd.crosstab(y_test, y_prediction,
                       rownames=['Actual'], 
                       colnames=['Predicted'], 
                       margins = True)

# Print the confusion matrix
print('Confusion Matrix')
print(conf_mat)

tp = conf_mat.iloc[0,0]
fp = conf_mat.iloc[1,0]
tn = conf_mat.iloc[1,1]
fn = conf_mat.iloc[0,1]


accuracy = (tn + tp)/(tn+ tp + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
sensitivity = tp/(tp + fn)
specificity = tn/(tn + fp)

print('\n')
print('Evaluation Metrics')
print('Accuracy: ', ((accuracy)*100).round(1), '%')
print('Precision: ', ((precision)*100).round(1), '%')
print('Recall: ', ((recall)*100).round(1), '%')
print('Sensitivity: ', ((sensitivity)*100).round(1), '%')
print('Specificity: ', ((specificity)*100).round(1), '%')

from sklearn.metrics import cohen_kappa_score

kappa = cohen_kappa_score(y_test, y_prediction)

print('Kappa: ', ((kappa)*100).round(1), '%',)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Confusion Matrix
Predicted    0    1  All
Actual                  
0          548   74  622
1          184   92  276
All        732  166  898


Evaluation Metrics
Accuracy:  71.3 %
Precision:  74.9 %
Recall:  88.1 %
Sensitivity:  88.1 %
Specificity:  33.3 %
Kappa:  24.1 %
